## 구멍 만들기_single_폴더별

In [1]:
import cv2
import numpy as np
import random
import matplotlib.pyplot as plt
import glob
import os

# 이미지 폴더 경로
input_folder = r'C:\hole\single'
output_folder = r'C:\hole\single_result'

# 구멍을 채울 이미지 (fill_image)
fill_image_path = 'fill.jpg'
fill_image = cv2.imread(fill_image_path)

# 이미지가 제대로 불러왔는지 확인
if fill_image is None:
    raise FileNotFoundError("채울 이미지를 불러올 수 없습니다. 경로를 확인하세요.")

# 폴더 내의 모든 이미지 파일 불러오기
image_paths = glob.glob(os.path.join(input_folder, '*.jpg'))

# 이미지 처리
for image_path in image_paths:
    # 이미지 불러오기
    image = cv2.imread(image_path)
    if image is None:
        print(f"이미지를 불러올 수 없습니다: {image_path}")
        continue

    # BGR 이미지를 HSV로 변환
    hsv_image = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)

    # 초록색 범위 설정 (HSV)
    lower_green = np.array([30, 40, 40])  # 초록색 하한 값
    upper_green = np.array([90, 255, 255])  # 초록색 상한 값

    # 초록색 영역 마스크 생성
    green_mask = cv2.inRange(hsv_image, lower_green, upper_green)

    # 초록색 영역의 좌표들 찾기
    green_coords = np.column_stack(np.where(green_mask > 0))

    # 구멍 개수 설정
    num_holes = 3  # 원하는 구멍 개수

    # 구멍이 생길 수 있는 영역을 제한 (가장자리 여백 추가)
    border_margin = 200  # 구멍이 가장자리에서 떨어져야 할 최소 거리

    # 구멍 생성
    result = image.copy()  # 결과 이미지 복사본 생성
    for _ in range(num_holes):
        if len(green_coords) == 0:
            print("초록색 영역이 없습니다.")
            break

        while True:
            # 초록색 영역에서 랜덤한 위치 선택
            coord_idx = random.randint(0, len(green_coords) - 1)
            center_y, center_x = green_coords[coord_idx]

            # 구멍이 가장자리에서 떨어진 위치에 있는지 확인
            if center_x > border_margin and center_x < image.shape[1] - border_margin and center_y > border_margin and center_y < image.shape[0] - border_margin:
                break  # 가장자리에서 충분히 떨어진 경우에만 구멍을 생성

        # 불규칙한 타원형 다각형 모양의 구멍을 생성
        num_vertices = random.randint(10, 20)  # 타원형 다각형 꼭짓점 개수
        axis_x = random.randint(40, 60)  # 타원의 가로축 반지름
        axis_y = random.randint(30, 50)  # 타원의 세로축 반지름
        angle_offset = random.uniform(0, 360)  # 타원의 회전 각도

        polygon_points = []
        valid_polygon = True  # 구멍이 초록색 영역 내에 있는지 확인하는 플래그

        for i in range(num_vertices):
            angle = 2 * np.pi * i / num_vertices + angle_offset  # 각도를 고르게 분포시키며 회전 추가
            r_x = axis_x + random.randint(-10, 20)  # 가로축 반경에 약간의 랜덤 변동 추가
            r_y = axis_y + random.randint(-10, 20)  # 세로축 반경에 약간의 랜덤 변동 추가
            x = int(center_x + r_x * np.cos(angle))
            y = int(center_y + r_y * np.sin(angle))

            # 구멍의 꼭짓점이 초록색 영역 내에 있는지 확인
            if green_mask[y, x] == 0:
                valid_polygon = False  # 구멍이 초록색을 벗어나면 false로 설정
                break

            polygon_points.append([x, y])

        if not valid_polygon:
            # 구멍이 초록색 영역을 벗어나면 다시 구멍을 생성
            continue

        polygon_points = np.array(polygon_points, np.int32).reshape((-1, 1, 2))

        # 다각형 마스크 생성
        mask = np.zeros_like(image)
        cv2.fillPoly(mask, [polygon_points], (255, 255, 255))

        # 다각형 영역 구하기
        x_min = np.min(polygon_points[:, 0, 0])
        x_max = np.max(polygon_points[:, 0, 0])
        y_min = np.min(polygon_points[:, 0, 1])
        y_max = np.max(polygon_points[:, 0, 1])

        # 채울 이미지를 다각형 크기에 맞게 자름
        fill_image_resized = cv2.resize(fill_image, (x_max - x_min, y_max - y_min))

        # 마스크로 다각형 이미지 적용
        fill_mask = np.zeros_like(fill_image_resized)
        cv2.fillPoly(fill_mask, [np.array([[(x - x_min, y - y_min) for x, y in polygon_points[:, 0]]], np.int32)], (255, 255, 255))

        # 다각형으로 잘린 이미지를 마스크로 적용
        fill_circle = cv2.bitwise_and(fill_image_resized, fill_mask)

        # 구멍 크기에 맞춰서 다각형 이미지를 채움
        region = result[y_min:y_max, x_min:x_max]
        final_result = np.where(mask[y_min:y_max, x_min:x_max] > 0, fill_circle, region)

        result[y_min:y_max, x_min:x_max] = final_result

    # 결과 이미지를 RGB로 변환하여 저장
    result_rgb = cv2.cvtColor(result, cv2.COLOR_BGR2RGB)

    # 출력 경로 지정 및 저장
    output_path = os.path.join(output_folder, os.path.basename(image_path))
    cv2.imwrite(output_path, cv2.cvtColor(result_rgb, cv2.COLOR_RGB2BGR))
    print(f"이미지를 '{output_path}'로 저장했습니다.")

이미지를 'C:\hole\single_result\IMG_6542.JPG'로 저장했습니다.
이미지를 'C:\hole\single_result\IMG_6543.JPG'로 저장했습니다.
이미지를 'C:\hole\single_result\IMG_6544.JPG'로 저장했습니다.
이미지를 'C:\hole\single_result\IMG_6545.JPG'로 저장했습니다.
이미지를 'C:\hole\single_result\IMG_6546.JPG'로 저장했습니다.
이미지를 'C:\hole\single_result\IMG_6547.JPG'로 저장했습니다.
이미지를 'C:\hole\single_result\IMG_6548.JPG'로 저장했습니다.
이미지를 'C:\hole\single_result\IMG_6549.JPG'로 저장했습니다.
이미지를 'C:\hole\single_result\IMG_6550.JPG'로 저장했습니다.
이미지를 'C:\hole\single_result\IMG_6551.JPG'로 저장했습니다.
이미지를 'C:\hole\single_result\IMG_6552.JPG'로 저장했습니다.
이미지를 'C:\hole\single_result\IMG_6553.JPG'로 저장했습니다.
이미지를 'C:\hole\single_result\IMG_6554.JPG'로 저장했습니다.
이미지를 'C:\hole\single_result\IMG_6555.JPG'로 저장했습니다.
이미지를 'C:\hole\single_result\IMG_6556.JPG'로 저장했습니다.
이미지를 'C:\hole\single_result\IMG_6557.JPG'로 저장했습니다.
이미지를 'C:\hole\single_result\IMG_6558.JPG'로 저장했습니다.
이미지를 'C:\hole\single_result\IMG_6559.JPG'로 저장했습니다.
이미지를 'C:\hole\single_result\IMG_6560.JPG'로 저장했습니다.
이미지를 'C:\hole\single_result\IMG

이미지를 'C:\hole\single_result\IMG_7078.JPG'로 저장했습니다.
이미지를 'C:\hole\single_result\IMG_7079.JPG'로 저장했습니다.
이미지를 'C:\hole\single_result\IMG_7080.JPG'로 저장했습니다.
이미지를 'C:\hole\single_result\IMG_7081.JPG'로 저장했습니다.
이미지를 'C:\hole\single_result\IMG_7082.JPG'로 저장했습니다.
이미지를 'C:\hole\single_result\IMG_7083.JPG'로 저장했습니다.
이미지를 'C:\hole\single_result\IMG_7084.JPG'로 저장했습니다.
이미지를 'C:\hole\single_result\IMG_7085.JPG'로 저장했습니다.
이미지를 'C:\hole\single_result\IMG_7086.JPG'로 저장했습니다.
이미지를 'C:\hole\single_result\IMG_7087.JPG'로 저장했습니다.
이미지를 'C:\hole\single_result\IMG_7088.JPG'로 저장했습니다.
이미지를 'C:\hole\single_result\IMG_7089.JPG'로 저장했습니다.
이미지를 'C:\hole\single_result\IMG_7090.JPG'로 저장했습니다.
이미지를 'C:\hole\single_result\IMG_7091.JPG'로 저장했습니다.
이미지를 'C:\hole\single_result\IMG_7092.JPG'로 저장했습니다.
이미지를 'C:\hole\single_result\IMG_7093.JPG'로 저장했습니다.
이미지를 'C:\hole\single_result\IMG_7094.JPG'로 저장했습니다.
이미지를 'C:\hole\single_result\IMG_7095.JPG'로 저장했습니다.
이미지를 'C:\hole\single_result\IMG_7096.JPG'로 저장했습니다.
이미지를 'C:\hole\single_result\IMG